In [ ]:
#Import necessary libraries
from PIL import Image
import numpy as np

In [ ]:
# Function to convert text to binary
def text_to_binary(text):
    """Convert string to binary representation"""
     # Convert each character to its 8-bit binary form and join them
    return ''.join(format(ord(char), '08b') for char in text)

In [ ]:
# Function to convert binary back to text
def binary_to_text(binary):
    """Convert binary string back to text"""
    # Split binary string into 8-bit chunks and convert each to a character
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join([chr(int(b, 2)) for b in chars])

In [ ]:
# Function to encode a message into an image using LSB steganography
def encode_image(img, message):
    """Hide message in image using LSB steganography"""
    img = img.convert('RGB')
    np_img = np.array(img) # Convert image to numpy array

# Convert message to binary and add a 16-bit delimiter at the end
    binary_message = text_to_binary(message) + '1111111111111111'  # New delimiter
    binary_index = 0
# Iterate over each pixel and channel to embed the message bits
    for row in np_img:
        for pixel in row:
            for channel in range(3):  # R, G, B channels
                if binary_index < len(binary_message):
                    # Set the LSB of the channel to the message bit
                    pixel[channel] = (int(pixel[channel]) & 0b11111110) | int(binary_message[binary_index])
                    binary_index += 1
    # If message is too long for the image, raise an error
    if binary_index < len(binary_message):
        raise ValueError("Message too long for this image.")
    # Convert numpy array back to image and return
    return Image.fromarray(np_img)

In [ ]:
# Function to decode a hidden message from an image
def decode_image(img):
    """Extract hidden message from image"""
    img = img.convert('RGB')
    np_img = np.array(img)

    binary_data = ''
    for row in np_img:
        for pixel in row:
            for channel in range(3):
                binary_data += str(pixel[channel] & 1)

    # Look for our 16-bit delimiter (0xFFFF)
    delimiter_index = binary_data.find('1111111111111111')
    
    if delimiter_index == -1:
        return "No message found or invalid format"
    
    message_bits = binary_data[:delimiter_index]
    
    # Convert binary to text
    chars = [message_bits[i:i+8] for i in range(0, len(message_bits), 8)]
    try:
        return ''.join([chr(int(b, 2)) for b in chars if len(b) == 8])
    except:
        return "Error decoding message"

In [ ]:
# Test the encode and decode functions

# Load an image (replace 'input.png' with your image file)
img = Image.open('input.png')

# Message to hide
secret_message = "Hello, Steganography!"

# Encode the message into the image
encoded_img = encode_image(img, secret_message)

# Save the encoded image (optional)
encoded_img.save('encoded_output.png')

# Load the encoded image for decoding
encoded_img_loaded = Image.open('encoded_output.png')

# Decode the hidden message
decoded_message = decode_image(encoded_img_loaded)

print("Original message:", secret_message)
print("Decoded message :", decoded_message)